In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
# from matplotlib import pyplot
%matplotlib inline

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
from sklearn import metrics
import random
from statistics import mean
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier, plot_importance
import os

In [3]:
num_classifiers = [1]

matplotlib.use('Agg')

output = []

# mergedfeatures1 = pd.read_csv('mergedfeaturessep102.csv')
mergedfeatures1 = pd.read_csv('corrected_permacts.csv')

# mergedfeatures = mergedfeatures1[['pkgname', 'lowest_android_version', 'highest_android_version', 'paid', 'file_size', 'max_downloads_log',
# 'developer_email', 'privacy_policy_link', 'developer_address', 'developer_website', 'days_since_last_update',
# 'DevRegisteredDomain', 'LenDescription', 'LenWhatsNew', 'ReviewsAverage', 'CurrentVersion',
# 'Genre', 'ContentRating', 'LastUpdated', 'LenTitle', 'AndroidVersion', 'DeveloperCategory', 'isSpamming',
# 'net', 'intent', 'bluetooth', 'app', 'provider', 'speech', 'nfc', 'media', 'hardware', 'google',
# 'os', 'CALENDAR', 'CAMERA', 'CONTACTS', 'LOCATION', 'MICROPHONE', 'PHONE', 'SENSORS',
# 'SMS', 'STORAGE', 'FourStarRatings', 'ThreeStarRatings', 'FiveStarRatings', 'OneStarRatings',
# 'TwoStarRatings', 'status']]
# mergedfeatures["FourStarRatings"] = mergedfeatures["FourStarRatings"].fillna(0)
# mergedfeatures["ThreeStarRatings"] = mergedfeatures["ThreeStarRatings"].fillna(0)
# mergedfeatures["FiveStarRatings"] = mergedfeatures["FiveStarRatings"].fillna(0)
# mergedfeatures["OneStarRatings"] = mergedfeatures["OneStarRatings"].fillna(0)
# mergedfeatures["TwoStarRatings"] = mergedfeatures["TwoStarRatings"].fillna(0)

mergedfeatures = mergedfeatures1[['pkgname', 'lowest_android_version', 'highest_android_version', 'paid', 'file_size',
'developer_email', 'privacy_policy_link', 'developer_address', 'developer_website',
'DevRegisteredDomain', 'LenDescription', 'CurrentVersion',
'Genre', 'ContentRating','LenTitle', 'AndroidVersion', 'DeveloperCategory', 'isSpamming',
'net', 'intent', 'bluetooth', 'app', 'provider', 'speech', 'nfc', 'media', 'hardware', 'google',
'os', 'CALENDAR', 'CAMERA', 'CONTACTS', 'LOCATION', 'MICROPHONE', 'PHONE', 'SENSORS',
'SMS', 'STORAGE', 'status']]


mergedfeatures = mergedfeatures.drop(['pkgname'], axis=1, inplace=False)
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ['lowest_android_version', 'highest_android_version', 'CurrentVersion',
                      'Genre', 'ContentRating', 'AndroidVersion', 'DeveloperCategory']


In [4]:
mergedfeatures = encode_and_bind(mergedfeatures, features_to_encode[0])
mergedfeatures = encode_and_bind(mergedfeatures, features_to_encode[1])
mergedfeatures = encode_and_bind(mergedfeatures, features_to_encode[2])
mergedfeatures = encode_and_bind(mergedfeatures, features_to_encode[3])
mergedfeatures = encode_and_bind(mergedfeatures, features_to_encode[4])
mergedfeatures = encode_and_bind(mergedfeatures, features_to_encode[5])
mergedfeatures = encode_and_bind(mergedfeatures, features_to_encode[6])

In [5]:
train, test = train_test_split(mergedfeatures, test_size=0.3, stratify=mergedfeatures.status)
output.append("Full - Train dataset: " + str(train.shape))
print("Full - Train dataset: " + str(train.shape))
output.append("Full - Test dataset: " + str(test.shape))
print("Full - Test dataset: " + str(test.shape))

#this method would help us select a balanced dataset of size k
def sampling_k_elements(group, k=50000):
    if len(group) < k:
        return group
    return group.sample(k)

#defininging and initilizing a number of lists to store the XGBoostClassifiers 
#and the random parameters
estimators_list = [256,512]
depths_list = [2,3]

#trying out different number of classifiers 1-15
# num_classifiers = [i for i in range(11,13, 2)]

Full - Train dataset: (609359, 2391)
Full - Test dataset: (261155, 2391)


In [6]:
#initialize the n classifiers with randomally selected parameters
# for n in num_classifiers:
output.append("Full - Trying " + str(num_classifiers[0]) +  " classifiers......")
print("Full - Trying " + str(num_classifiers[0]) +  " classifiers......")
classifiers_list = []
for c in range(num_classifiers[0]):
  nest = random.choice(estimators_list)
  mxd = random.choice(depths_list)
  classifier=XGBClassifier(n_estimators=nest, max_depth=mxd)
  classifiers_list.append(classifier)

Full - Trying 1 classifiers......


In [7]:
for classifier in classifiers_list:
  #train the classifiers on balanced datasets drawn from the training dataset
  pos = train[train['status'] == 1].sample(50000, replace = False)
  neg = train[train['status'] == 0].sample(50000, replace = False)
  # balanced_set = pos.append(neg)
  balanced_set = pd.concat([pos, neg])
  #balanced_set = train.groupby('status').apply(sampling_k_elements).reset_index(drop=True)
  output.append("Full - Balanced dataset: " + str(balanced_set.shape))
  print("Full - Balanced dataset: " + str(balanced_set.shape))
  #X,y = balanced_set.iloc[:, :-1], balanced_set.iloc[:, -1]
  y = balanced_set['status'].values
  X = balanced_set.drop(['status'],axis=1,inplace=False).values
  classifier.fit(X,y)
#print("classifier", classifiers_list.index(classifier), "   :", classifier)

Full - Balanced dataset: (100000, 2391)


In [8]:
# draw a new sample from the training dataset, while maintaining the original 
# class distribution, validation dataset, same size like test dataset
# test2 is validation set, X2 is validation as well
train_2 , test_2 = train_test_split(train, test_size=0.4285, stratify=train.status)
output.append("Full - Validation dataset: " + str(test_2.shape))
print("Full - Validation dataset: " + str(test_2.shape))
y2 = test_2['status'].values

Full - Validation dataset: (261111, 2391)


In [9]:
print(test_2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 261111 entries, 69788 to 629437
Columns: 2391 entries, paid to DeveloperCategory_Moderate
dtypes: bool(2360), int64(31)
memory usage: 651.4 MB
None


In [10]:
X2 = test_2.drop(['status'],axis=1,inplace=False).values
#print("X2 shape: ", X2.shape)
#print("y2 shape: ", y2.shape)

In [11]:
size = X2.shape[0]
target = np.zeros((size,2))
size2 = len(mergedfeatures.columns)-1
importaccum = np.zeros(size2)
#size = X2.shape[0]
#print("size : " , size)
#target = [0.0]*size
for clasfier in classifiers_list:
  preds = clasfier.predict_proba(X2)
  target = np.add(target, preds) 
  #preds = clasfier.predict(X2)
  #print("Before: target[1]:", target[1])
  #target = [target[i] + preds[i] for i in range(len(target))]
  #print("After: target[1]:", target[1])
  #plot important features
  importances = clasfier.feature_importances_
  importaccum = np.add(importaccum,importances)

In [12]:
features = mergedfeatures.columns
importaccum2 = importaccum / len(classifiers_list)
indices = np.argsort(importaccum2)
  # customized number 
num_features = 20
plt.figure(figsize=(20,10))
plt.title('Feature Importances')
# only plot the customized number of features
plt.barh(range(num_features), importaccum2[indices[-num_features:]], color='b', align='center')
plt.yticks(range(num_features), [features[i] for i in indices[-num_features:]])
plt.xlabel('Relative Importance')
pltname = str(len(classifiers_list)) + "-classifiers-Evaluation-Full"
#plt.savefig("%s.png" % pltname, dpi=300) #2400x1800 pixels instead of 800x600, try 600
#   plt.savefig("%s.pdf" % pltname, dpi=800)
# plt.cla()

In [13]:
#target2 = [target[i]/len(classifiers_list) for i in range(len(target))]
target2 = target / len(classifiers_list)
# calculate AUC
auc = roc_auc_score(y2, target2[:,1])
output.append('Full - Evaluation AUC: %.3f' % auc)
print('Full - Evaluation AUC: %.3f' % auc)

# fpr means false-positive-rate
# tpr means true-positive-rate
fpr, tpr, _ = metrics.roc_curve(y2, target2[:,1])
auc_score = metrics.auc(fpr, tpr)

Full - Evaluation AUC: 0.757


In [14]:
# get tp, fp, tn, fn for validation dataset
indices_fp = (y2 == 0) & (np.round(target2[:,1]) == 1)
indices_fn = (y2 == 1) & (np.round(target2[:,1]) == 0)
indices_tp = (y2 == 1) & (np.round(target2[:,1]) == 1)
indices_tn = (y2 == 0) & (np.round(target2[:,1]) == 0)

: 

In [15]:
y3 = test['status'].values
X3 = test.drop(['status'],axis=1,inplace=False).values

size2 = X3.shape[0]
target1 = np.zeros((size2,2))
for clasfier in classifiers_list:
  preds2 = clasfier.predict_proba(X3)
  target1 = np.add(target1, preds2) 

target3 = target1 / len(classifiers_list)
# calculate AUC
auc2 = roc_auc_score(y3, target3[:,1])
output.append('Full - Testing AUC: %.3f' % auc2)
print('Full - Testing AUC: %.3f' % auc2)

# fpr means false-positive-rate
# tpr means true-positive-rate
fpr2, tpr2, _ = metrics.roc_curve(y3, target3[:,1])
auc_score2 = metrics.auc(fpr2, tpr2)

In [ ]:
# get tp, fp, tn, fn for test dataset
indices_fp_test = (y3 == 0) & (np.round(target1[:,1]) == 1)
indices_fn_test = (y3 == 1) & (np.round(target1[:,1]) == 0)
indices_tp_test = (y3 == 1) & (np.round(target1[:,1]) == 1)
indices_tn_test = (y3 == 0) & (np.round(target1[:,1]) == 0)